In [9]:
# import libraries
import geopandas as gpd 
import pandas as pd
from shapely import ops
import os
import shapely
import numpy as np
import psycopg2
import pandana as pdna
import networkx as nx
import multiprocessing as mp
import matplotlib.pyplot as plt
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *

from shapely.geometry import *
shapely.speedups.enable()

%matplotlib inline 

In [10]:
# add creating netwotk graph code
def create_graph(gdf, precision=3, simplify=1):
    '''Create a networkx DiGraph given a GeoDataFrame of lines. Every line will
    correspond to two directional graph edges, one forward, one reverse. The
    original line row and direction will be stored in each edge. Every node
    will be where endpoints meet (determined by being very close together) and
    will store a clockwise ordering of incoming edges.
    '''
    # The geometries sometimes have tiny end parts - get rid of those!
    gdf.geometry = gdf.geom.simplify(simplify)

    #G = nx.DiGraph()
    G = nx.Graph()

    # TODO: converting to string is probably unnecessary - keeping float may be
    # faster
    def make_node(coord, precision):
        return tuple(np.round(coord, precision))

    # Edges are stored as (from, to, data), where from and to are nodes.
    # az1 is the azimuth of the first segment of the geometry (point into the
    # geometry), az2 is for the last segment (pointing out of the geometry)
    def add_edges(row, G):
        geom = row.geom
        coords = list(geom.coords)
        geom_r = LineString(coords[::-1])
        coords_r = geom_r.coords
        start = make_node(coords[0], precision)
        end = make_node(coords[-1], precision)
        # Add forward edge
        fwd_attr ={}
        for k,v in row.items():
            fwd_attr[k]=v
        fwd_attr['forward']= 1
        fwd_attr['geometry']=  geom
        fwd_attr['length']=  geom.length
#         fwd_attr['az1']=  azimuth_cartesian(coords[0], coords[1])
#         fwd_attr['az2']=  azimuth_cartesian(coords[-2], coords[-1])
        fwd_attr['visited']= 0

        G.add_edge(start, end, **fwd_attr)

    gdf.apply(add_edges, axis=1, args=[G])

    return G

In [3]:
# database connection
con = psycopg2.connect(database="walkability", user="postgres", password=1234,
    host="****",port="5432")
pednet= gpd.read_postgis('SELECT * FROM public.pednet100m',con,crs={'init': 'epsg:2019'})
pednet['distance'] = pednet.geom.length
pednet['distance']

0         74.097267
1         86.664696
2         63.990661
3         63.990661
4         98.384042
5         90.022112
6         90.022112
7         90.022112
8         52.952604
9         52.952604
10        61.841117
11        61.841117
12        91.898827
13        18.823190
14        33.944756
15        83.179815
16        83.179815
17        78.401199
18        89.854370
19        50.401166
20        50.401166
21        86.633939
22        86.633939
23        86.633939
24        97.680349
25        90.409356
26        80.588990
27        80.588990
28        63.082536
29        63.082536
            ...    
125232    78.554208
125233    78.554208
125234    60.643137
125235    78.866441
125236    57.266962
125237    57.266962
125238    79.303550
125239    79.303550
125240    84.928817
125241    59.765053
125242    94.719212
125243    65.043230
125244    65.366956
125245    50.990459
125246    50.990459
125247    92.031043
125248    53.693961
125249    68.807735
125250    68.807735


In [11]:
# creating network graph
G = create_graph(pednet)
#from G to urbanaccess network model - one node idx,x,y one edge from to weight
#G.edges(data=True)

In [ ]:
#degree calculation
degrees = [val for (node, val) in G.degree()]

In [ ]:
##G centrality calculation
import time
start = time.time()
deg_centrality = nx.degree_centrality(G)
a_list = []
b_list = []
for node, val in sorted(deg_centrality.items(), key=lambda x: x[1],  reverse=True):
     a_list.append(node)
     b_list.append(val)
     df = pd.DataFrame({'Node': a_list, 'Degree': b_list})
del a_list, b_list
    #print(node, val)
end = time.time()
df.to_csv("./deg_centrality.csv")
print(end - start)


In [ ]:
#closeness_centrality 19160.19558262825
import time
start = time.time()
closeness_centrality = nx.closeness_centrality(G)
a_list = []
b_list = []
for node, val in sorted(closeness_centrality.items(), key=lambda x: x[1],  reverse=True):
     a_list.append(node)
     b_list.append(val)
     df1 = pd.DataFrame({'Node': a_list, 'closeness_centrality': b_list})

    
del a_list, b_list
    #print(node, val)
end = time.time()
print(end - start)

In [ ]:
#betweenness_centrality
import time
start = time.time()
betweenness_centrality = nx.betweenness_centrality(G)
a_list = []
b_list = []
for node, val in sorted(betweenness_centrality.items(), key=lambda x: x[1],  reverse=True):
     a_list.append(node)
     b_list.append(val)
     df2 = pd.DataFrame({'Node': a_list, 'betweenness_centrality': b_list})

    
del a_list, b_list
    #print(node, val)
end = time.time()
print(end - start)
df1.to_csv("./closeness_centrality.csv")

In [ ]:
# multiprocessing betweenness if necessary
from multiprocessing import Pool
import itertools
def partitions(nodes, n):
    "Partitions the nodes into n subsets"
    nodes_iter = iter(nodes)
    while True:
        partition = tuple(itertools.islice(nodes_iter,n))
        if not partition:
            return
        yield partition
def between_parallel(G, processes = None):
    p = Pool(processes=processes)
    part_generator = 4*len(p._pool)
    node_partitions = list(partitions(G.nodes(), int(len(G)/part_generator)))
    num_partitions = len(node_partitions)
 
    bet_map = p.map(btwn_pool,
                        zip([G]*num_partitions,
                        [True]*num_partitions,
                        [None]*num_partitions,
                        node_partitions))
 
    bt_c = bet_map[0]
    for bt in bet_map[1:]:
        for n in bt:
            bt_c[n] += bt[n]
    return bt_c
bt = between_parallel(G)
#bt.to_csv("./betweenness.csv")
betweenness= pd.DataFrame.from_dict(bt, orient='index').to_csv("./betweenness1.csv", index=False)


In [12]:
# get network "from" and "to" from nodes
edges = nx.to_pandas_edgelist(G,'from','to')
to = edges['to'].tolist()
fr = edges['from'].tolist()
fr = list(set(fr))
to = list(set(to))
to.extend(fr)
nodes = list(set(to))
nodes = pd.DataFrame(nodes)
nodes.columns=['x', 'y']
nodes['xy'] = nodes.apply(lambda z: (z.x,z.y),axis=1)


In [46]:
# save nodes
nodes.to_csv("./nodes.csv")

In [21]:
# multi processing network nodes and edges index creation
import pandas as pd
import numpy as np
from multiprocessing import Pool
import time
start = time.time()
num_partitions = 16 #number of partitions to split dataframe
num_cores = 16 #number of cores on your machine



def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def set_node_id(edges):
    nodes = pd.read_csv('./nodes.csv')
    edges = pd.read_csv('./edges.csv')
    for ix, node in nodes.iterrows():
        indicies = edges[edges.to == node.xy].index
        edges.loc[indicies,'to'] = ix
        indicies = edges[edges['from'] == node.xy].index
        edges.loc[indicies,'from'] = ix
    return edges
    
    
edges = parallelize_dataframe(edges, set_node_id)
end = time.time()
print(end - start)


In [ ]:
#Initialize the Pandana network 
#https://github.com/gboeing/urban-data-science/blob/master/20-Accessibility-Walkability/pandana-accessibility-demo-simple.ipynb
# if there is an error, please run this before creating pdna network
edges["to"] = edges["to"].astype(int)
edges["from"] = edges["from"].astype(int)


In [ ]:
# creating pandana network
import pandana as pdna
import pandas as pd
from pandana import Network

transit_ped_net = pdna.Network(nodes["x"],
                               nodes["y"],
                               edges["from"],
                               edges["to"],
                               pd.DataFrame([edges1.length]).T,
                               twoway=True)

transit_ped_net.save_hdf5('/media/DATADRIVE/walkability/ped_net_final_0525.hd5')

In [35]:
transit_ped_net.precompute(10000)

In [ ]:
# read the network from saved file
transit_ped_net = pdna.Network.from_hdf5('/media/DATADRIVE/walkability/ped_net_final_0525.hd5')

In [ ]:
# load data from pg database total 23 layers
con = psycopg2.connect(database="walkability", user="postgres", password=****,
    host="****",port="5432")
#service layers
foodconv = gpd.read_postgis('SELECT * FROM staging.food_convenience_store',con,crs={'init': 'epsg:2019'})
fastfood = gpd.read_postgis('SELECT * FROM staging.food_fastfood',con,crs={'init': 'epsg:2019'})
supermarket = gpd.read_postgis('SELECT * FROM staging.food_supermarket',con,crs={'init': 'epsg:2019'})
hospital = gpd.read_postgis('SELECT * FROM staging.health_hospital',con,crs={'init': 'epsg:2019'})
sxhclinic = gpd.read_postgis('SELECT * FROM staging.health_sexual_health_clinic',con,crs={'init': 'epsg:2019'})
wkclinic = gpd.read_postgis('SELECT * FROM staging.health_walkin',con,crs={'init': 'epsg:2019'})
arena = gpd.read_postgis('SELECT * FROM staging.publicservice_arena',con,crs={'init': 'epsg:2019'})
daycare = gpd.read_postgis('SELECT * FROM staging.publicservice_day_care_centre',con,crs={'init': 'epsg:2019'})
dropin = gpd.read_postgis('SELECT * FROM staging.publicservice_dropin',con,crs={'init': 'epsg:2019'})
ems = gpd.read_postgis('SELECT * FROM staging.publicservice_ems',con,crs={'init': 'epsg:2019'})
famcentre = gpd.read_postgis('SELECT * FROM staging.publicservice_family_resource_centre',con,crs={'init': 'epsg:2019'})
fire = gpd.read_postgis('SELECT * FROM staging.publicservice_fire_facility',con,crs={'init': 'epsg:2019'})
library = gpd.read_postgis('SELECT * FROM staging.publicservice_library',con,crs={'init': 'epsg:2019'})
police = gpd.read_postgis('SELECT * FROM staging.publicservice_police_facility',con,crs={'init': 'epsg:2019'})
recreation = gpd.read_postgis('SELECT * FROM staging.publicservice_recreation',con,crs={'init': 'epsg:2019'})
school = gpd.read_postgis('SELECT * FROM staging.publicservice_school',con,crs={'init': 'epsg:2019'})
thingstodo = gpd.read_postgis('SELECT * FROM staging.publicservice_thingstodo',con,crs={'init': 'epsg:2019'})
voteloc = gpd.read_postgis('SELECT * FROM staging.publicservice_voting_location',con,crs={'init': 'epsg:2019'})
bike_parking = gpd.read_postgis('SELECT * FROM staging.transit_bicycle_parking_on_street',con,crs={'init': 'epsg:2019'})
bike_station = gpd.read_postgis('SELECT * FROM staging.transit_bicycle_station_indoor',con,crs={'init': 'epsg:2019'})
ttc_station = gpd.read_postgis('SELECT * FROM staging.transit_ttc_station',con,crs={'init': 'epsg:2019'})
ttc_accessible = gpd.read_postgis('SELECT * FROM staging.transit_ttc_station_accessible',con,crs={'init': 'epsg:2019'})
ttc_stop = gpd.read_postgis('SELECT * FROM staging.transit_ttc_stop',con,crs={'init': 'epsg:2019'})
ts = gpd.read_postgis('SELECT * FROM staging.traffic_signals',con,crs={'init': 'epsg:2019'})
buildings = gpd.read_postgis('SELECT * FROM public.buildings_blank',con,crs={'init': 'epsg:2019'})

In [ ]:
# keep wanted columns
foodconv=foodconv[['gid', 'x', 'y','name','type']]
fastfood=fastfood[['gid', 'x', 'y','name','type']]
supermarket=supermarket[['gid', 'x', 'y','name','type']]
hospital=hospital[['gid', 'x', 'y','name','type']]
sxhclinic=sxhclinic[['gid', 'x', 'y','name','ward']]
wkclinic=wkclinic[['gid', 'x', 'y','name','type']]
school=school[['gid', 'x', 'y','name','school_type']]
arena=arena[['gid', 'x', 'y','name','community']]
daycare=daycare[['gid', 'x', 'y','name','place_name']]
dropin=dropin[['gid', 'x', 'y','name','facility']]
ems=ems[['gid', 'x', 'y','name','place_name']]
famcentre=famcentre[['gid', 'x', 'y','name','agency']]
fire=fire[['gid', 'x', 'y','name','address']]
library=library[['gid', 'x', 'y','name','address_full']]
police=police[['gid', 'x', 'y','name','address']]
recreation=recreation[['gid', 'x', 'y','rc_name','rc_type']]
thingstodo=thingstodo[['gid', 'x', 'y','name','td_type']]
voteloc=voteloc[['gid', 'x', 'y','name','linear_nam']]
bike_parking=bike_parking[['gid', 'x', 'y','address_fu','parking_ty']]
bike_station=bike_station[['gid', 'x', 'y','address_fu','station_ty']]
ttc_station=ttc_station[['gid', 'x', 'y','name','pt_type']]
ttc_stop=ttc_stop[['gid', 'x', 'y','name','stop_code']]
ttc_accessible=ttc_accessible[['gid', 'x', 'y','name','pt_type']]
ts=ts[['gid', 'x', 'y','main','side_1']]
buildings= buildings[['ctuid', 'ctname', 'ctnum', 'geotext', 'geom', 'gid']]

In [37]:
#get node_ids for points for each layer

x, y = foodconv.x, foodconv.y
foodconv["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(foodconv["node_ids"], name="foodconv")

x, y = fastfood.x, fastfood.y
fastfood["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(fastfood["node_ids"], name="fastfood")

#supermarket
x, y = supermarket.x, supermarket.y
supermarket["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(supermarket["node_ids"], name="supermarket")

x, y = hospital.x, hospital.y
hospital["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(hospital["node_ids"], name="hospital")

x, y = sxhclinic.x, sxhclinic.y
sxhclinic["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(sxhclinic["node_ids"], name="sxhclinic")

x, y = wkclinic.x, wkclinic.y
wkclinic["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(wkclinic["node_ids"], name="wkclinic")

x, y = school.x, school.y
school["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(school["node_ids"], name="school")

x, y = arena.x, arena.y
arena["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(arena["node_ids"], name="arena")

x, y = daycare.x, daycare.y
daycare["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(daycare["node_ids"], name="daycare")

x, y = dropin.x, dropin.y
dropin["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(dropin["node_ids"], name="dropin")

x, y = ems.x, ems.y
ems["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(ems["node_ids"], name="ems")

x, y = famcentre.x, famcentre.y
famcentre["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(famcentre["node_ids"], name="famcentre")

x, y = fire.x, fire.y
fire["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(fire["node_ids"], name="fire")

x, y = library.x, library.y
library["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(library["node_ids"], name="library")

x, y = police.x, police.y
police["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(police["node_ids"], name="police")

x, y = recreation.x, recreation.y
recreation["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(recreation["node_ids"], name="recreation")

x, y = thingstodo.x, thingstodo.y
thingstodo["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(thingstodo["node_ids"], name="thingstodo")

x, y = voteloc.x, voteloc.y
voteloc["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(voteloc["node_ids"], name="voteloc")

x, y = bike_parking.x, bike_parking.y
bike_parking["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(bike_parking["node_ids"], name="bike_parking")

x, y = bike_station.x, bike_station.y
bike_station["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(bike_station["node_ids"], name="bike_station")

x, y = ttc_stop.x, ttc_stop.y
ttc_stop["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(ttc_stop["node_ids"], name="ttc_stop")

x, y = ttc_station.x, ttc_station.y
ttc_station["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(ttc_station["node_ids"], name="ttc_station")

x, y = ttc_accessible.x, ttc_accessible.y
ttc_accessible["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(ttc_accessible["node_ids"], name="ttc_accessible")

x, y = ts.x, ts.y
ts["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(ts["node_ids"], name="ts")

x, y = buildings.geom.centroid.x, buildings.geom.centroid.y
buildings["node_ids"] = transit_ped_net.get_node_ids(x, y)

/opt/anaconda3/lib/python3.6/site-packages/pandana/network.py:370: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  distances, indexes = self.kdtree.query(xys.as_matrix())


In [ ]:
#get nearest 2 points with id
n=2
maxdistance = 5000 

transit_ped_net.set_pois("foodconv", maxdistance , n, foodconv.x, foodconv.y)
foodconv_walk_distances = transit_ped_net.nearest_pois(maxdistance , "foodconv", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("fastfood", maxdistance , n, fastfood.x, fastfood.y)
fastfood_walk_distances = transit_ped_net.nearest_pois(maxdistance , "fastfood", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("supermarket", maxdistance , n, supermarket.x, supermarket.y)
supermarket_walk_distances = transit_ped_net.nearest_pois(maxdistance , "supermarket", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("hospital", maxdistance , n, hospital.x, hospital.y)
hospital_walk_distances = transit_ped_net.nearest_pois(maxdistance , "hospital", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("sxhclinic", maxdistance , n, sxhclinic.x, sxhclinic.y)
sxhclinic_walk_distances = transit_ped_net.nearest_pois(maxdistance , "sxhclinic", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("wkclinic", maxdistance , n, wkclinic.x, wkclinic.y)
wkclinic_walk_distances = transit_ped_net.nearest_pois(maxdistance , "wkclinic", num_pois=n, include_poi_ids=True)


transit_ped_net.set_pois("school", maxdistance , n, school.x, school.y)
school_walk_distances = transit_ped_net.nearest_pois(maxdistance , "school", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("arena", maxdistance , n, arena.x, arena.y)
arena_walk_distances = transit_ped_net.nearest_pois(maxdistance , "arena", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("daycare", maxdistance , n, daycare.x, daycare.y)
daycare_walk_distances = transit_ped_net.nearest_pois(maxdistance , "daycare", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("dropin", maxdistance , n, dropin.x, dropin.y)
dropin_walk_distances = transit_ped_net.nearest_pois(maxdistance , "dropin", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("ems", maxdistance , n, ems.x, ems.y)
ems_walk_distances = transit_ped_net.nearest_pois(maxdistance , "ems", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("famcentre", maxdistance , n, famcentre.x, famcentre.y)
famcentre_walk_distances = transit_ped_net.nearest_pois(maxdistance , "famcentre", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("fire", maxdistance , n, fire.x, fire.y)
fire_walk_distances = transit_ped_net.nearest_pois(maxdistance , "fire", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("library", maxdistance , n, library.x, library.y)
library_walk_distances = transit_ped_net.nearest_pois(maxdistance , "library", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("police", maxdistance , n, police.x, police.y)
police_walk_distances = transit_ped_net.nearest_pois(maxdistance , "police", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("recreation", maxdistance , n, recreation.x, recreation.y)
recreation_walk_distances = transit_ped_net.nearest_pois(maxdistance , "recreation", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("thingstodo", maxdistance , n, thingstodo.x, thingstodo.y)
thingstodo_walk_distances = transit_ped_net.nearest_pois(maxdistance , "thingstodo", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("voteloc", maxdistance , n, voteloc.x, voteloc.y)
voteloc_walk_distances = transit_ped_net.nearest_pois(maxdistance , "voteloc", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("bike_parking", maxdistance , n, bike_parking.x, bike_parking.y)
bike_parking_walk_distances = transit_ped_net.nearest_pois(maxdistance , "bike_parking", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("bike_station", maxdistance , n, bike_station.x, bike_station.y)
bike_station_walk_distances = transit_ped_net.nearest_pois(maxdistance , "bike_station", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("ttc_stop", maxdistance , n, ttc_stop.x, ttc_stop.y)
ttc_stop_walk_distances = transit_ped_net.nearest_pois(maxdistance , "ttc_stop", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("ttc_station", maxdistance , n, ttc_station.x, ttc_station.y)
ttc_station_walk_distances = transit_ped_net.nearest_pois(maxdistance , "ttc_station", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("ttc_accessible", maxdistance , n, ttc_accessible.x, ttc_accessible.y)
ttc_accessible_walk_distances = transit_ped_net.nearest_pois(maxdistance , "ttc_accessible", num_pois=n, include_poi_ids=True)

transit_ped_net.set_pois("ts", maxdistance , n, ts.x, ts.y)
ts_walk_distances = transit_ped_net.nearest_pois(maxdistance , "ts", num_pois=n, include_poi_ids=True)

In [ ]:
# keep wanted columns
foodconv.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
fastfood.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
supermarket.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
hospital.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
sxhclinic.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
wkclinic.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
school.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
arena.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
daycare.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
dropin.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
ems.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
famcentre.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
fire.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
library.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
police.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
recreation.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
thingstodo.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
voteloc.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
bike_parking.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
bike_station.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
ttc_station.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
ttc_stop.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
ttc_accessible.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)
ts.drop(['x', 'y', 'node_ids'], axis=1, inplace=True)

In [ ]:
# rename columns
n=2
columns =  ['d_fc_'+str(i) for i in range(0,n,1)]
columns.extend(['n_fc_'+str(i) for i in range(0,n,1)])
foodconv_walk_distances.columns = columns

columns =  ['d_ff_'+str(i) for i in range(0,n,1)]
columns.extend(['n_ff_'+str(i) for i in range(0,n,1)])
fastfood_walk_distances.columns = columns

columns =  ['d_sm_'+str(i) for i in range(0,n,1)]
columns.extend(['n_sm_'+str(i) for i in range(0,n,1)])
supermarket_walk_distances.columns = columns

columns =  ['d_hp_'+str(i) for i in range(0,n,1)]
columns.extend(['n_hp_'+str(i) for i in range(0,n,1)])
hospital_walk_distances.columns = columns

columns =  ['d_sxhc_'+str(i) for i in range(0,n,1)]
columns.extend(['n_sxhc_'+str(i) for i in range(0,n,1)])
sxhclinic_walk_distances.columns = columns

columns =  ['d_wc_'+str(i) for i in range(0,n,1)]
columns.extend(['n_wc_'+str(i) for i in range(0,n,1)])
wkclinic_walk_distances.columns = columns

columns =  ['d_sc_'+str(i) for i in range(0,n,1)]
columns.extend(['n_sc_'+str(i) for i in range(0,n,1)])
school_walk_distances.columns = columns

columns =  ['d_ar_'+str(i) for i in range(0,n,1)]
columns.extend(['n_ar_'+str(i) for i in range(0,n,1)])
arena_walk_distances.columns = columns

columns =  ['d_dc_'+str(i) for i in range(0,n,1)]
columns.extend(['n_dc_'+str(i) for i in range(0,n,1)])
daycare_walk_distances.columns = columns

columns =  ['d_di_'+str(i) for i in range(0,n,1)]
columns.extend(['n_di_'+str(i) for i in range(0,n,1)])
dropin_walk_distances.columns = columns

columns =  ['d_ems_'+str(i) for i in range(0,n,1)]
columns.extend(['n_ems_'+str(i) for i in range(0,n,1)])
ems_walk_distances.columns = columns

columns =  ['d_frc_'+str(i) for i in range(0,n,1)]
columns.extend(['n_frc_'+str(i) for i in range(0,n,1)])
famcentre_walk_distances.columns = columns

columns =  ['d_fr_'+str(i) for i in range(0,n,1)]
columns.extend(['n_fr_'+str(i) for i in range(0,n,1)])
fire_walk_distances.columns = columns

columns =  ['d_lb_'+str(i) for i in range(0,n,1)]
columns.extend(['n_lb_'+str(i) for i in range(0,n,1)])
library_walk_distances.columns = columns

columns =  ['d_plc_'+str(i) for i in range(0,n,1)]
columns.extend(['n_plc_'+str(i) for i in range(0,n,1)])
police_walk_distances.columns = columns

columns =  ['d_rec_'+str(i) for i in range(0,n,1)]
columns.extend(['n_rec_'+str(i) for i in range(0,n,1)])
recreation_walk_distances.columns = columns

columns =  ['d_thing_'+str(i) for i in range(0,n,1)]
columns.extend(['n_thing_'+str(i) for i in range(0,n,1)])
thingstodo_walk_distances.columns = columns

columns =  ['d_vtl_'+str(i) for i in range(0,n,1)]
columns.extend(['n_vtl_'+str(i) for i in range(0,n,1)])
voteloc_walk_distances.columns = columns

columns =  ['d_bp_'+str(i) for i in range(0,n,1)]
columns.extend(['n_bp_'+str(i) for i in range(0,n,1)])
bike_parking_walk_distances.columns = columns

columns =  ['d_bs_'+str(i) for i in range(0,n,1)]
columns.extend(['n_bs_'+str(i) for i in range(0,n,1)])
bike_station_walk_distances.columns = columns

columns =  ['d_ttcstop_'+str(i) for i in range(0,n,1)]
columns.extend(['n_ttcstop_'+str(i) for i in range(0,n,1)])
ttc_stop_walk_distances.columns = columns

columns =  ['d_ttcst_'+str(i) for i in range(0,n,1)]
columns.extend(['n_ttcst_'+str(i) for i in range(0,n,1)])
ttc_station_walk_distances.columns = columns

columns =  ['d_ttcacc_'+str(i) for i in range(0,n,1)]
columns.extend(['n_ttcacc_'+str(i) for i in range(0,n,1)])
ttc_accessible_walk_distances.columns = columns

columns =  ['d_ts_'+str(i) for i in range(0,n,1)]
columns.extend(['n_ts_'+str(i) for i in range(0,n,1)])
ts_walk_distances.columns = columns

In [43]:
#merging walk distances with buildings data
c = 0
l = len(buildings.node_ids.unique())
for ix,group in buildings.groupby('node_ids'):
    group
    c +=1

    for j in range(0,n,1):
        
        buildings.loc[group.index,'d_fc_{}'.format(j)] = foodconv_walk_distances.loc[ix]['d_fc_{}'.format(j)]
        buildings.loc[group.index,'n_fc_{}'.format(j)] =foodconv_walk_distances.loc[ix]['n_fc_{}'.format(j)]
        
        buildings.loc[group.index,'d_ff_{}'.format(j)] = fastfood_walk_distances.loc[ix]['d_ff_{}'.format(j)]
        buildings.loc[group.index,'n_ff_{}'.format(j)] = fastfood_walk_distances.loc[ix]['n_ff_{}'.format(j)] 
      
        buildings.loc[group.index,'d_sm_{}'.format(j)] = supermarket_walk_distances.loc[ix]['d_sm_{}'.format(j)]
        buildings.loc[group.index,'n_sm_{}'.format(j)] = supermarket_walk_distances.loc[ix]['n_sm_{}'.format(j)]
        
        buildings.loc[group.index,'d_hp_{}'.format(j)] = hospital_walk_distances.loc[ix]['d_hp_{}'.format(j)]
        buildings.loc[group.index,'n_hp_{}'.format(j)] = hospital_walk_distances.loc[ix]['n_hp_{}'.format(j)]
        
        buildings.loc[group.index,'d_sxhc_{}'.format(j)] = sxhclinic_walk_distances.loc[ix]['d_sxhc_{}'.format(j)]
        buildings.loc[group.index,'n_sxhc_{}'.format(j)] = sxhclinic_walk_distances.loc[ix]['n_sxhc_{}'.format(j)]
        
        buildings.loc[group.index,'d_wc_{}'.format(j)] = wkclinic_walk_distances.loc[ix]['d_wc_{}'.format(j)]
        buildings.loc[group.index,'n_wc_{}'.format(j)] = wkclinic_walk_distances.loc[ix]['n_wc_{}'.format(j)]
        
        buildings.loc[group.index,'d_sc_{}'.format(j)] = school_walk_distances.loc[ix]['d_sc_{}'.format(j)]
        buildings.loc[group.index,'n_sc_{}'.format(j)] = school_walk_distances.loc[ix]['n_sc_{}'.format(j)]
        
        buildings.loc[group.index,'d_ar_{}'.format(j)] = arena_walk_distances.loc[ix]['d_ar_{}'.format(j)]
        buildings.loc[group.index,'n_ar_{}'.format(j)] = arena_walk_distances.loc[ix]['n_ar_{}'.format(j)]
        
        buildings.loc[group.index,'d_dc_{}'.format(j)] = daycare_walk_distances.loc[ix]['d_dc_{}'.format(j)]
        buildings.loc[group.index,'n_dc_{}'.format(j)] = daycare_walk_distances.loc[ix]['n_dc_{}'.format(j)]
        
        buildings.loc[group.index,'d_di_{}'.format(j)] = dropin_walk_distances.loc[ix]['d_di_{}'.format(j)]
        buildings.loc[group.index,'n_di_{}'.format(j)] = dropin_walk_distances.loc[ix]['n_di_{}'.format(j)]
        
        buildings.loc[group.index,'d_ems_{}'.format(j)] = ems_walk_distances.loc[ix]['d_ems_{}'.format(j)]
        buildings.loc[group.index,'n_ems_{}'.format(j)] = ems_walk_distances.loc[ix]['n_ems_{}'.format(j)]
        
        buildings.loc[group.index,'d_frc_{}'.format(j)] = famcentre_walk_distances.loc[ix]['d_frc_{}'.format(j)]
        buildings.loc[group.index,'n_frc_{}'.format(j)] = famcentre_walk_distances.loc[ix]['n_frc_{}'.format(j)]
        
        buildings.loc[group.index,'d_fr_{}'.format(j)] = fire_walk_distances.loc[ix]['d_fr_{}'.format(j)]
        buildings.loc[group.index,'n_fr_{}'.format(j)] = fire_walk_distances.loc[ix]['n_fr_{}'.format(j)]
        
        buildings.loc[group.index,'d_lb_{}'.format(j)] = library_walk_distances.loc[ix]['d_lb_{}'.format(j)]
        buildings.loc[group.index,'n_lb_{}'.format(j)] = library_walk_distances.loc[ix]['n_lb_{}'.format(j)]
        
        buildings.loc[group.index,'d_plc_{}'.format(j)] = police_walk_distances.loc[ix]['d_plc_{}'.format(j)]
        buildings.loc[group.index,'n_plc_{}'.format(j)] = police_walk_distances.loc[ix]['n_plc_{}'.format(j)]
        
        buildings.loc[group.index,'d_rec_{}'.format(j)] = recreation_walk_distances.loc[ix]['d_rec_{}'.format(j)]
        buildings.loc[group.index,'n_rec_{}'.format(j)] = recreation_walk_distances.loc[ix]['n_rec_{}'.format(j)]
        
        buildings.loc[group.index,'d_thing_{}'.format(j)] = thingstodo_walk_distances.loc[ix]['d_thing_{}'.format(j)]
        buildings.loc[group.index,'n_thing_{}'.format(j)] = thingstodo_walk_distances.loc[ix]['n_thing_{}'.format(j)]
        
        buildings.loc[group.index,'d_vtl_{}'.format(j)] = voteloc_walk_distances.loc[ix]['d_vtl_{}'.format(j)]
        buildings.loc[group.index,'n_vtl_{}'.format(j)] = voteloc_walk_distances.loc[ix]['n_vtl_{}'.format(j)]
        
        buildings.loc[group.index,'d_bp_{}'.format(j)] = bike_parking_walk_distances.loc[ix]['d_bp_{}'.format(j)]
        buildings.loc[group.index,'n_bp_{}'.format(j)] = bike_parking_walk_distances.loc[ix]['n_bp_{}'.format(j)]
        
        buildings.loc[group.index,'d_bs_{}'.format(j)] = bike_station_walk_distances.loc[ix]['d_bs_{}'.format(j)]
        buildings.loc[group.index,'n_bs_{}'.format(j)] = bike_station_walk_distances.loc[ix]['n_bs_{}'.format(j)]
        
        buildings.loc[group.index,'d_ttcstop_{}'.format(j)] = ttc_stop_walk_distances.loc[ix]['d_ttcstop_{}'.format(j)]
        buildings.loc[group.index,'n_ttcstop_{}'.format(j)] = ttc_stop_walk_distances.loc[ix]['n_ttcstop_{}'.format(j)]
        
        buildings.loc[group.index,'d_ttcst_{}'.format(j)] = ttc_station_walk_distances.loc[ix]['d_ttcst_{}'.format(j)]
        buildings.loc[group.index,'n_ttcst_{}'.format(j)] = ttc_station_walk_distances.loc[ix]['n_ttcst_{}'.format(j)]
        
        buildings.loc[group.index,'d_ttcacc_{}'.format(j)] = ttc_accessible_walk_distances.loc[ix]['d_ttcacc_{}'.format(j)]
        buildings.loc[group.index,'n_ttcacc_{}'.format(j)] = ttc_accessible_walk_distances.loc[ix]['n_ttcacc_{}'.format(j)]
        
        buildings.loc[group.index,'d_ts_{}'.format(j)] = ttc_accessible_walk_distances.loc[ix]['d_ts_{}'.format(j)]
        buildings.loc[group.index,'n_ts_{}'.format(j)] = ttc_accessible_walk_distances.loc[ix]['n_ts_{}'.format(j)]

In [ ]:
# calculate minutes fields based on distances. This can be done by adding this process to loop in the above cell.
#This method is faster than above cell. It takes 95 seconds
import time
start = time.time()


buildings['m_fc_0'] = buildings.apply(lambda row: row.d_fc_0/(1.2*60), axis=1)
buildings['m_fc_1'] = buildings.apply(lambda row: row.d_fc_1/(1.2*60), axis=1)

buildings['m_ff_0'] = buildings.apply(lambda row: row.d_ff_0/(1.2*60), axis=1)
buildings['m_ff_1'] = buildings.apply(lambda row: row.d_ff_1/(1.2*60), axis=1)

buildings['m_sm_0'] = buildings.apply(lambda row: row.d_sm_0/(1.2*60), axis=1)
buildings['m_sm_1'] = buildings.apply(lambda row: row.d_sm_1/(1.2*60), axis=1)

buildings['m_hp_0'] = buildings.apply(lambda row: row.d_hp_0/(1.2*60), axis=1)
buildings['m_hp_1'] = buildings.apply(lambda row: row.d_hp_1/(1.2*60), axis=1)

buildings['m_sxhc_0'] = buildings.apply(lambda row: row.d_sxhc_0/(1.2*60), axis=1)
buildings['m_sxhc_1'] = buildings.apply(lambda row: row.d_sxhc_1/(1.2*60), axis=1)

buildings['m_wc_0'] = buildings.apply(lambda row: row.d_wc_0/(1.2*60), axis=1)
buildings['m_wc_1'] = buildings.apply(lambda row: row.d_wc_1/(1.2*60), axis=1)


buildings['m_sc_0'] = buildings.apply(lambda row: row.d_sc_0/(1.2*60), axis=1)
buildings['m_sc_1'] = buildings.apply(lambda row: row.d_sc_1/(1.2*60), axis=1)

buildings['m_ar_0'] = buildings.apply(lambda row: row.d_ar_0/(1.2*60), axis=1)
buildings['m_ar_1'] = buildings.apply(lambda row: row.d_ar_1/(1.2*60), axis=1)

buildings['m_dc_0'] = buildings.apply(lambda row: row.d_dc_0/(1.2*60), axis=1)
buildings['m_dc_1'] = buildings.apply(lambda row: row.d_dc_1/(1.2*60), axis=1)

buildings['m_di_0'] = buildings.apply(lambda row: row.d_di_0/(1.2*60), axis=1)
buildings['m_di_1'] = buildings.apply(lambda row: row.d_di_1/(1.2*60), axis=1)

buildings['m_ems_0'] = buildings.apply(lambda row: row.d_ems_0/(1.2*60), axis=1)
buildings['m_ems_1'] = buildings.apply(lambda row: row.d_ems_1/(1.2*60), axis=1)

buildings['m_frc_0'] = buildings.apply(lambda row: row.d_frc_0/(1.2*60), axis=1)
buildings['m_frc_1'] = buildings.apply(lambda row: row.d_frc_1/(1.2*60), axis=1)


buildings['m_fr_0'] = buildings.apply(lambda row: row.d_fr_0/(1.2*60), axis=1)
buildings['m_fr_1'] = buildings.apply(lambda row: row.d_fr_1/(1.2*60), axis=1)

buildings['m_lb_0'] = buildings.apply(lambda row: row.d_lb_0/(1.2*60), axis=1)
buildings['m_lb_1'] = buildings.apply(lambda row: row.d_lb_1/(1.2*60), axis=1)

buildings['m_plc_0'] = buildings.apply(lambda row: row.d_plc_0/(1.2*60), axis=1)
buildings['m_plc_1'] = buildings.apply(lambda row: row.d_plc_1/(1.2*60), axis=1)

buildings['m_rec_0'] = buildings.apply(lambda row: row.d_rec_0/(1.2*60), axis=1)
buildings['m_rec_1'] = buildings.apply(lambda row: row.d_rec_1/(1.2*60), axis=1)

buildings['m_thing_0'] = buildings.apply(lambda row: row.d_thing_0/(1.2*60), axis=1)
buildings['m_thing_1'] = buildings.apply(lambda row: row.d_thing_1/(1.2*60), axis=1)

buildings['m_vtl_0'] = buildings.apply(lambda row: row.d_vtl_0/(1.2*60), axis=1)
buildings['m_vtl_1'] = buildings.apply(lambda row: row.d_vtl_1/(1.2*60), axis=1)

buildings['m_bp_0'] = buildings.apply(lambda row: row.d_bp_0/(1.2*60), axis=1)
buildings['m_bp_1'] = buildings.apply(lambda row: row.d_bp_1/(1.2*60), axis=1)

buildings['m_bs_0'] = buildings.apply(lambda row: row.d_bs_0/(1.2*60), axis=1)
buildings['m_bs_1'] = buildings.apply(lambda row: row.d_bs_1/(1.2*60), axis=1)

buildings['m_ttcstop_0'] = buildings.apply(lambda row: row.d_ttcstop_0/(1.2*60), axis=1)
buildings['m_ttcstop_1'] = buildings.apply(lambda row: row.d_ttcstop_1/(1.2*60), axis=1)

buildings['m_ttcst_0'] = buildings.apply(lambda row: row.d_ttcst_0/(1.2*60), axis=1)
buildings['m_ttcst_1'] = buildings.apply(lambda row: row.d_ttcst_1/(1.2*60), axis=1)

buildings['m_ttcacc_0'] = buildings.apply(lambda row: row.d_ttcacc_0/(1.2*60), axis=1)
buildings['m_ttcacc_1'] = buildings.apply(lambda row: row.d_ttcacc_1/(1.2*60), axis=1)

buildings['m_ts_0'] = buildings.apply(lambda row: row.d_ts_0/(1.2*60), axis=1)
buildings['m_ts_1'] = buildings.apply(lambda row: row.d_ts_1/(1.2*60), axis=1)
end = time.time()
print(end - start)

In [ ]:
# keep wanted columns
foodconv=foodconv[['gid', 'name','type']]
fastfood=fastfood[['gid', 'name','type']]
supermarket=supermarket[['gid', 'name','type']]
hospital=hospital[['gid', 'name','type']]
sxhclinic=sxhclinic[['gid', 'name','ward']]
wkclinic=wkclinic[['gid', 'name','type']]
school=school[['gid', 'name','school_type']]
arena=arena[['gid', 'name','community']]
daycare=daycare[['gid', 'name','place_name']]
dropin=dropin[['gid', 'name','facility']]
ems=ems[['gid', 'name','place_name']]
famcentre=famcentre[['gid', 'name','agency']]
fire=fire[['gid', 'name','address']]
library=library[['gid', 'name','address_full']]
police=police[['gid', 'name','address']]
recreation=recreation[['gid', 'rc_name','rc_type']]
thingstodo=thingstodo[['gid', 'name','td_type']]
voteloc=voteloc[['gid', 'name','linear_nam']]
bike_parking=bike_parking[['gid', 'address_fu','parking_ty']]
bike_station=bike_station[['gid', 'address_fu','station_ty']]
ttc_station=ttc_station[['gid', 'name','pt_type']]
ttc_stop=ttc_stop[['gid', 'name','stop_code']]
ttc_accessible=ttc_accessible[['gid', 'name','pt_type']]
ts=ts[['gid', 'main','side_1']]

In [ ]:
#merge data takes 1065.7354395389557 seconds
import time
start = time.time()
# rename the first point columns
buildings = pd.merge(buildings, foodconv, how='left', left_on=['n_fc_0'], right_on = ['gid'] )
buildings = buildings.rename(columns={'name': 'n_fc_0n', 'type': 'n_fc_0type'})

buildings = pd.merge(buildings, fastfood, how='left', left_on=['n_ff_0'], right_on = ['gid'] )
buildings = buildings.rename(columns={'name': 'n_ff_0n', 'type': 'n_ff_0type'})

buildings = pd.merge(buildings, supermarket, how='left', left_on=['n_sm_0'], right_on = ['gid'] )
buildings = buildings.rename(columns={'name': 'n_sm_0n', 'type': 'n_sm_0type'})

buildings = pd.merge(buildings, hospital, how='left', left_on=['n_hp_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_hp_0n', 'type': 'n_hp_0type'})

buildings = pd.merge(buildings, sxhclinic, how='left', left_on=['n_sxhc_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_sxhc_0n', 'ward': 'n_sxhc_0type'})

buildings = pd.merge(buildings, wkclinic, how='left', left_on=['n_wc_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_wc_0n', 'type': 'n_wc_0type'})

buildings = pd.merge(buildings, school, how='left', left_on=['n_sc_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_sc_0n', 'school_type': 'n_sc_0type'})

buildings = pd.merge(buildings, arena, how='left', left_on=['n_ar_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_ar_0n', 'community': 'n_ar_0type'})

buildings = pd.merge(buildings, library, how='left', left_on=['n_lb_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_lb_0n', 'address_full': 'n_lb_0type'})

buildings = pd.merge(buildings, daycare, how='left', left_on=['n_dc_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_dc_0n', 'place_name': 'n_dc_0type'})

buildings = pd.merge(buildings, fire, how='left', left_on=['n_fr_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_fr_0n', 'address': 'n_fr_0type'})

buildings = pd.merge(buildings, dropin, how='left', left_on=['n_di_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_di_0n', 'facility': 'n_di_0type'})

buildings = pd.merge(buildings, ems, how='left', left_on=['n_ems_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_ems_0n', 'place_name': 'n_ems_0type'})

buildings = pd.merge(buildings, famcentre, how='left', left_on=['n_frc_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_frc_0n', 'agency': 'n_frc_0type'})

buildings = pd.merge(buildings, police, how='left', left_on=['n_plc_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_plc_0n', 'address': 'n_plc_0type'})

buildings = pd.merge(buildings, recreation, how='left', left_on=['n_rec_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'rc_name': 'n_rec_0n', 'rc_type': 'n_rec_0type'})

buildings = pd.merge(buildings, thingstodo, how='left', left_on=['n_thing_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_thing_0n', 'td_type': 'n_thing_0type'})

buildings = pd.merge(buildings, voteloc, how='left', left_on=['n_vtl_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_vtl_0n', 'linear_nam': 'n_vtl_0type'})

buildings = pd.merge(buildings, bike_parking, how='left', left_on=['n_bp_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'adsress_fu': 'n_bp_0n', 'parking_ty': 'n_bp_0type'})

buildings = pd.merge(buildings, bike_station, how='left', left_on=['n_bs_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'adsress_fu': 'n_bs_0n', 'station_ty': 'n_bs_0type'})

buildings = pd.merge(buildings, ttc_stop, how='left', left_on=['n_ttcstop_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_ttcstop_0n', 'stop_code': 'n_ttcstop_0type'})

buildings = pd.merge(buildings, ttc_station, how='left', left_on=['n_ttcst_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_ttcst_0n', 'pt_type': 'n_ttcst_0type'})

buildings = pd.merge(buildings, ttc_accessible, how='left', left_on=['n_ttcacc_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_ttcacc_0n', 'pt_type': 'n_ttcacc_0type'})

buildings = pd.merge(buildings, ts, how='left', left_on=['n_ts_0'], right_on = ['gid']) 
buildings = buildings.rename(columns={'main': 'n_ts_0n', 'side_1': 'n_ts_0type'})

# rename the second point columns

buildings = pd.merge(buildings, foodconv, how='left', left_on=['n_fc_1'], right_on = ['gid'] )
buildings = buildings.rename(columns={'name': 'n_fc_1n', 'type': 'n_fc_1type'})

buildings = pd.merge(buildings, fastfood, how='left', left_on=['n_ff_1'], right_on = ['gid'] )
buildings = buildings.rename(columns={'name': 'n_ff_1n', 'type': 'n_ff_1type'})

buildings = pd.merge(buildings, supermarket, how='left', left_on=['n_sm_1'], right_on = ['gid'] )
buildings = buildings.rename(columns={'name': 'n_sm_1n', 'type': 'n_sm_1type'})

buildings = pd.merge(buildings, hospital, how='left', left_on=['n_hp_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_hp_1n', 'type': 'n_hp_1type'})

buildings = pd.merge(buildings, sxhclinic, how='left', left_on=['n_sxhc_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_sxhc_1n', 'ward': 'n_sxhc_1type'})

buildings = pd.merge(buildings, wkclinic, how='left', left_on=['n_wc_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_wc_1n', 'type': 'n_wc_1type'})

buildings = pd.merge(buildings, school, how='left', left_on=['n_sc_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_sc_1n', 'school_type': 'n_sc_1type'})

buildings = pd.merge(buildings, arena, how='left', left_on=['n_ar_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_ar_1n', 'community': 'n_ar_1type'})

buildings = pd.merge(buildings, library, how='left', left_on=['n_lb_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_lb_1n', 'address_full': 'n_lb_1type'})

buildings = pd.merge(buildings, daycare, how='left', left_on=['n_dc_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_dc_1n', 'place_name': 'n_dc_1type'})

buildings = pd.merge(buildings, fire, how='left', left_on=['n_fr_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_fr_1n', 'address': 'n_fr_1type'})

buildings = pd.merge(buildings, dropin, how='left', left_on=['n_di_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_di_1n', 'facility': 'n_di_1type'})

buildings = pd.merge(buildings, ems, how='left', left_on=['n_ems_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_ems_1n', 'place_name': 'n_ems_1type'})

buildings = pd.merge(buildings, famcentre, how='left', left_on=['n_frc_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_frc_1n', 'agency': 'n_frc_1type'})

buildings = pd.merge(buildings, police, how='left', left_on=['n_plc_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_plc_1n', 'address': 'n_plc_1type'})

buildings = pd.merge(buildings, recreation, how='left', left_on=['n_rec_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'rc_name': 'n_rec_1n', 'rc_type': 'n_rec_1type'})

buildings = pd.merge(buildings, thingstodo, how='left', left_on=['n_thing_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_thing_1n', 'td_type': 'n_thing_1type'})

buildings = pd.merge(buildings, voteloc, how='left', left_on=['n_vtl_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_vtl_1n', 'linear_nam': 'n_vtl_1type'})

buildings = pd.merge(buildings, bike_parking, how='left', left_on=['n_bp_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'adsress_fu': 'n_bp_1n', 'parking_ty': 'n_bp_1type'})

buildings = pd.merge(buildings, bike_station, how='left', left_on=['n_bs_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'adsress_fu': 'n_bs_1n', 'station_ty': 'n_bs_1type'})

buildings = pd.merge(buildings, ttc_stop, how='left', left_on=['n_ttcstop_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_ttcstop_1n', 'stop_code': 'n_ttcstop_1type'})

buildings = pd.merge(buildings, ttc_station, how='left', left_on=['n_ttcst_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_ttcst_1n', 'pt_type': 'n_ttcst_1type'})

buildings = pd.merge(buildings, ttc_accessible, how='left', left_on=['n_ttcacc_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'name': 'n_ttcacc_1n', 'pt_type': 'n_ttcacc_1type'})

buildings = pd.merge(buildings, ts, how='left', left_on=['n_ts_1'], right_on = ['gid']) 
buildings = buildings.rename(columns={'main': 'n_ts_1n', 'side_1': 'n_ts_1type'})


end = time.time()
print(end - start)

In [ ]:
# see all the columns
print(buildings.columns.values)

In [ ]:
#at last drop unwanted columns baased on the columns
buildings.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'gid_x', 'node_ids', 'stype_x', 'gid_y', 'x_y', 'y_y',
                'stype_y', 'gid_x.1', 'x_x.1', 'y_x.1', 'stype_x.1', 'gid_y.1', 'x_y.1', 'y_y.1', 'gid_x.2', 'x_x.2', 'y_x.2', 'gid_y.2',
                'x_y.2', 'y_y.2', 'gid_x.3',  'school_type_x', 'gid_y.3', 'gid_x.4', 'x_x.3', 'y_x.3', 'x_y.3', 'y_y.3', 'gid_y.4',
                'gid_x.5', 'x_x.4', 'y_x.4', 'gid_y.5' , 'x_y.4', 'y_y.4', 'gid_x.6', 'x_x.5', 'y_x.5' , 'gid_y.6', 'x_y.5', 'y_y.5',
                'gid_x.7', 'x_x.6', 'y_x.6', 'gid_y.7', 'x_y.6', 'y_y.6', 'gid_x.8', 'name_x', 'x_x.7', 'y_x.7',  'gid_y.8', 'x_y.7',
                'y_y.7',  'name_y', 'gid_x.9', 'x_x.8', 'y_x.8', 'gid_y.9', 'address_fu_x', 'name_x.1', 'x_y.8', 'y_y.8',  'gid_x.10',
                'address_fu_y', 'name_y.1', 'x_x.9', 'y_x.9', 'x_y.9', 'y_y.9', 'gid_y.10', 'x_x.10', 'y_x.10', 'gid_x.11', 'x_y.10',
                'y_y.10',  'gid_y.11', 'x_x.11', 'y_x.11', 'stype_y.1', 'gid_x.12', 'x_y.11', 'y_y.11', 'stype_x.2', 'gid_y.12', 'x_x.12',
                'y_x.12', 'stype_y.2', 'gid_x.13', 'x_y.12', 'y_y.12',  'gid_y.13',  'x_x.13', 'y_x.13', 'gid_x.14', 'x_y.13', 'y_y.13',
                'gid_y.14', 'school_type_y', 'gid_x.15', 'gid_y.15', 'x_x.14', 'y_x.14', 'community', 'gid_x.16', 'x_y.14', 'y_y.14',
                'x_x.15', 'y_x.15', 'gid_y.16', 'gid_x.17', 'x_y.15', 'y_y.15', 'gid_y.17', 'x_x.16', 'y_x.16', 'gid_x.18', 'x_y.16',
                'y_y.16', 'gid_y.18', 'x_x.17', 'y_x.17', 'gid_x.19', 'x_y.17', 'y_y.17', 'gid_y.19', 'name_x.2', 'x_x.18', 'y_x.18',
                'gid_x.20', 'x_y.18', 'y_y.18', 'name_y.2', 'gid_y.20', 'x_x.19', 'y_x.19', 'gid_x.21', 'address_fu_x.1', 'name_x.3',
                'x_y.19', 'y_y.19', 'gid_y.21', 'address_fu_y.1', 'name_y.3', 'x_x.20', 'y_x.20', 'x_y.20', 'y_y.20', 'gid_x.22', 'x_x.21',
                'y_x.21', 'gid_y.22', 'x_y.21', 'y_y.21', 'gid'], axis=1, inplace=True)

In [ ]:
print(buildings.columns.values)
['Unnamed: 0' 'ctuid' 'ctname' 'ctnum' 'geotext' 'geom' 'node_ids'
 'd_fc_0' 'n_fc_0' 'd_ff_0' 'n_ff_0' 'd_sm_0' 'n_sm_0' 'd_hp_0' 'n_hp_0'
 'd_sxhc_0' 'n_sxhc_0' 'd_wc_0' 'n_wc_0' 'd_sc_0' 'n_sc_0' 'd_ar_0'
 'n_ar_0' 'd_dc_0' 'n_dc_0' 'd_di_0' 'n_di_0' 'd_ems_0' 'n_ems_0' 'd_fc_1'
 'n_fc_1' 'd_ff_1' 'n_ff_1' 'd_sm_1' 'n_sm_1' 'd_hp_1' 'n_hp_1' 'd_sxhc_1'
 'n_sxhc_1' 'd_wc_1' 'n_wc_1' 'd_sc_1' 'n_sc_1' 'd_ar_1' 'n_ar_1' 'd_dc_1'
 'n_dc_1' 'd_di_1' 'n_di_1' 'd_ems_1' 'n_ems_1' 'd_frc_0' 'n_frc_0'
 'd_fr_0' 'n_fr_0' 'd_lb_0' 'n_lb_0' 'd_plc_0' 'n_plc_0' 'd_rec_0'
 'n_rec_0' 'd_thing_0' 'n_thing_0' 'd_vtl_0' 'n_vtl_0' 'd_bp_0' 'n_bp_0'
 'd_bs_0' 'n_bs_0' 'd_ttcstop_0' 'n_ttcstop_0' 'd_ttcst_0' 'n_ttcst_0'
 'd_ttcacc_0' 'n_ttcacc_0' 'd_ts_0' 'n_ts_0' 'd_frc_1' 'n_frc_1' 'd_fr_1'
 'n_fr_1' 'd_lb_1' 'n_lb_1' 'd_plc_1' 'n_plc_1' 'd_rec_1' 'n_rec_1'
 'd_thing_1' 'n_thing_1' 'd_vtl_1' 'n_vtl_1' 'd_bp_1' 'n_bp_1' 'd_bs_1'
 'n_bs_1' 'd_ttcstop_1' 'n_ttcstop_1' 'd_ttcst_1' 'n_ttcst_1' 'd_ttcacc_1'
 'n_ttcacc_1' 'd_ts_1' 'n_ts_1' 'm_fc_0' 'm_fc_1' 'm_ff_0' 'm_ff_1'
 'm_sm_0' 'm_sm_1' 'm_hp_0' 'm_hp_1' 'm_sxhc_0' 'm_sxhc_1' 'm_wc_0'
 'm_wc_1' 'm_sc_0' 'm_sc_1' 'm_ar_0' 'm_ar_1' 'm_dc_0' 'm_dc_1' 'm_di_0'
 'm_di_1' 'm_ems_0' 'm_ems_1' 'm_frc_0' 'm_frc_1' 'm_fr_0' 'm_fr_1'
 'm_lb_0' 'm_lb_1' 'm_plc_0' 'm_plc_1' 'm_rec_0' 'm_rec_1' 'm_thing_0'
 'm_thing_1' 'm_vtl_0' 'm_vtl_1' 'm_bp_0' 'm_bp_1' 'm_bs_0' 'm_bs_1'
 'm_ttcstop_0' 'm_ttcstop_1' 'm_ttcst_0' 'm_ttcst_1' 'm_ttcacc_0'
 'm_ttcacc_1' 'm_ts_0' 'm_ts_1' 'n_fc_0n' 'n_fc_0type' 'n_ff_0n'
 'n_ff_0type' 'n_sm_0n' 'n_sm_0type' 'n_hp_0n' 'n_hp_0type' 'n_sxhc_0n'
 'n_sxhc_0type' 'n_wc_0n' 'n_wc_0type' 'n_sc_0n' 'n_sc_0type' 'n_ar_0n'
 'n_ar_0type' 'n_lb_0n' 'n_lb_0type' 'n_dc_0n' 'n_dc_0type' 'n_fr_0n'
 'n_fr_0type' 'n_di_0.1' 'n_dc_0type.1' 'n_ems_0n' 'n_ems_0type'
 'n_frc_0n' 'n_frc_0type' 'n_plc_0n' 'n_plc_0type' 'n_rec_0n'
 'n_rec_0type' 'n_thing_0n' 'n_thing_0type' 'n_vtl_0n' 'n_vtl_0type'
 'n_bp_0type' 'n_bs_0type' 'n_ttcstop_0n' 'n_ttcstop_0type' 'n_ttcst_0n'
 'n_ttcst_0type' 'n_ttcacc_0n' 'n_ttcacc_0type' 'n_ts_0n' 'n_ts_0type'
 'n_fc_1n' 'n_fc_1type' 'n_ff_1n' 'n_ff_1type' 'n_sm_1n' 'n_sm_1type'
 'n_hp_1n' 'n_hp_1type' 'n_sxhc_1n' 'n_sxhc_1type' 'n_wc_1n' 'n_wc_1type'
 'n_sc_1n' 'n_sc_1type' 'n_ar_1n' 'n_ar_1type' 'n_lb_1n' 'n_lb_1type'
 'n_dc_1n' 'n_dc_1type' 'n_fr_1n' 'n_fr_1type' 'n_di_1.1' 'n_dc_1type.1'
 'n_ems_1n' 'n_ems_1type' 'n_frc_1n' 'n_frc_1type' 'n_plc_1n'
 'n_plc_1type' 'n_rec_1n' 'n_rec_1type' 'n_thing_1n' 'n_thing_1type'
 'n_vtl_1n' 'n_vtl_1type' 'n_bp_1type' 'n_bs_1type' 'n_ttcstop_1n'
 'n_ttcstop_1type' 'n_ttcst_1n' 'n_ttcst_1type' 'n_ttcacc_1n'
 'n_ttcacc_1type' 'gid' 'n_ts_1n' 'n_ts_1type']

In [44]:
# Creating SQLAlchemy's engine to use
engine = create_engine('postgresql://postgres:****@****:5432/walkability')

buildings['geometry'] = buildings['geom'].apply(lambda x: WKTElement(x.wkt, srid=2019))

#drop the geometry column as it is now duplicative
blds= buildings.drop('geom', 1, inplace=False)

# Use 'dtype' to specify column's type
# For the geom column, we will use GeoAlchemy's type 'Geometry'
blds.to_sql("buildings527", engine, if_exists='replace', index=False, 
                         dtype={'geometry': Geometry('MULTIPOLYGON', srid= 2019)})